In [8]:
import numpy as np
from tqdm import tqdm
import copy
from Step1_mar import mar_cd,mar_iter_ols,mar_PROJ,mar_adalasso,mar_scad,mar_lasso
from Step2_vector import vector_adalasso, vector_lasso, vector_scad
from Step3_sparse_mar import final_mar_cd,final_mar_adalasso,final_mar_adalasso2
from utils import rate,metrics_dif_kron,matrix_stack,genetate_data

## 时间序列样本生成

In [28]:
p = 2
dim_A = 3
dim_B = 3
T = 100
A_true,B_true,X = genetate_data(dim_A,dim_B,T,p,p_1=0.5,uniform=[-1,1],uniform_x=[1,10])
# X = matrix_stack(X, p)
U1, Sigma1, V1 = np.linalg.svd(A_true)
U2, Sigma2, V2 = np.linalg.svd(B_true)
V3, Sigma3, V3 = np.linalg.svd(np.kron(B_true,A_true))
print(max(abs((Sigma1))<1))
print(max(abs((Sigma2))<1))
print(max(abs((Sigma3))<1))
print(A_true)
print(B_true)

True
True
True
[[ 0.21349036 -0.90892363 -0.0938591   0.12524292  0.01241109 -0.        ]
 [ 0.          0.15162193 -0.22367464  0.         -0.          0.85397302]
 [ 0.         -0.         -0.21552404  0.50486657 -0.         -0.        ]]
[[ 0.         -0.          0.         -0.          0.         -0.        ]
 [ 0.56551944  0.          0.          0.          0.         -0.76721149]
 [ 0.          0.         -0.         -0.          0.         -0.42834358]]


## 第一阶段估计

In [29]:
A_cd, B_cd = mar_cd(X,p)
A_proj, B_proj = mar_PROJ(X,p)
A_iter_ols, B_iter_ols = mar_iter_ols(X,p)
print("MAR-坐标下降法")
print('张量积误差')
for i in range(p):
    print(metrics_dif_kron(A_true[:,i*dim_A:(i+1)*dim_A],B_true[:,i*dim_B:(i+1)*dim_B],A_hat=A_cd[:,i*dim_A:(i+1)*dim_A],B_hat=B_cd[:,i*dim_B:(i+1)*dim_B]))

print('-----------------------------------------')
print("MAR-投影法")
print('张量积误差')
for i in range(p):
    print(metrics_dif_kron(A_true[:,i*dim_A:(i+1)*dim_A],B_true[:,i*dim_B:(i+1)*dim_B],A_hat=A_proj[:,i*dim_A:(i+1)*dim_A],B_hat=B_proj[:,i*dim_B:(i+1)*dim_B]))

print('-----------------------------------------')
print("MAR-迭代最小二乘法")
print('张量积误差')
for i in range(p):
    print(metrics_dif_kron(A_true[:,i*dim_A:(i+1)*dim_A],B_true[:,i*dim_B:(i+1)*dim_B],A_hat=A_iter_ols[:,i*dim_A:(i+1)*dim_A],B_hat=B_iter_ols[:,i*dim_B:(i+1)*dim_B]))



100%|██████████| 5000/5000 [00:01<00:00, 4117.16it/s]

MAR-坐标下降法
张量积误差
-2.4738653692344132
-0.3950424324512826
-----------------------------------------
MAR-投影法
张量积误差
-1.1460619569789758
-0.14550714473892212
-----------------------------------------
MAR-迭代最小二乘法
张量积误差
-2.1037884071859003
-1.8935864631673265


In [32]:
A_cd = A_iter_ols
B_cd = B_iter_ols

## 第二阶段参数选择
#### 改进方法

In [33]:
lambda_ = 1
A_hat_adalasso,B_hat_adalasso = vector_adalasso(X,A_cd,B_cd,p,lam=lambda_)
# A_hat_lasso,B_hat_lasso = vector_lasso(X,A_cd,B_cd,lam)
# A_hat_scad,B_hat_scad = vector_scad(X,A_cd,B_cd,lam)

print('转换模型 adaptive lasso')
rate(A_true,B_true,A_hat_adalasso,B_hat_adalasso)
print('--'*20)

# print('转换模型 lasso')
# rate(A_true,B_true.T,A_hat_lasso,B_hat_lasso)
# print('--'*20)

# print('转换模型 scad ')
# rate(A_true,B_true[0,:,:].T,A_hat_scad,B_hat_scad)


  0%|          | 10/5000 [00:00<02:42, 30.71it/s]

转换模型 adaptive lasso
rate_A_mar_0: 0.875
rate_B_mar_0: 0.6
rate_A_mar_1: 0.6
rate_B_mar_1: 1.0
rate_mar_0: 0.6956521739130435
rate_mar_1: 0.6923076923076923
----------------------------------------


#### 原模型方法

In [34]:
A_hat_1, B_hat_1 = mar_adalasso(X,A_cd,B_cd,p,lam1=1,lam2=1,max_iter=5000)
# print(np.where(A_hat_==0,0,1))
# print(np.where(B_hat_==0,0,1))
print('mar adaptive lasso')
rate(A_true,B_true,A_hat_1,B_hat_1)
print('张量积误差')
for i in range(p):
    print(metrics_dif_kron(A_true[:,i*dim_A:(i+1)*dim_A],B_true[:,i*dim_B:(i+1)*dim_B],A_hat=A_hat_1[:,i*dim_A:(i+1)*dim_A],B_hat=B_hat_1[:,i*dim_B:(i+1)*dim_B]))

# print(metrics_dif_kron(A_true,B_true.T,A_hat=A_hat_1,B_hat=B_hat_1))
# print("--"*20)

# A_hat_2, B_hat_2 = mar_lasso(X,A_cd,B_cd,lam1=1,lam2=1)
# print('mar lasso')
# rate(A_true,B_true,A_hat_2,B_hat_2)
# print('张量积误差')
# print(metrics_dif_kron(A_true,B_true,A_hat_2,B_hat_2))
# print("--"*20)

# A_hat_3, B_hat_3 = mar_scad(X,A_cd,B_cd,lam1=1,lam2=1)
# print('mar scad')
# rate(A_true,B_true,A_hat_3,B_hat_3)
# print('张量积误差')
# print(metrics_dif_kron(A_true,B_true,A_hat_3,B_hat_3))
# print("--"*20)

  0%|          | 2/5000 [00:00<00:41, 119.38it/s]

mar adaptive lasso
rate_A_mar_0: 1.0
rate_B_mar_0: 0.5333333333333333
rate_A_mar_1: 0.2
rate_B_mar_1: 0.6666666666666666
rate_mar_0: 0.6956521739130435
rate_mar_1: 0.3076923076923077
张量积误差
-1.1400211974591792
-0.44446481502585067


## 第三阶段

In [22]:
A_hat1, B_hat1 = final_mar_cd(X,p,A_hat_adalasso,B_hat_adalasso,A_cd,B_cd,max_iter=5000)
print(A_hat1)
print(B_hat1)
for i in range(p):
    print(metrics_dif_kron(A_true[:,i*dim_A:(i+1)*dim_A],B_true[:,i*dim_B:(i+1)*dim_B],A_hat=A_hat1[:,i*dim_A:(i+1)*dim_A],B_hat=B_hat1[:,i*dim_B:(i+1)*dim_B]))


  0%|          | 7/5000 [00:00<00:32, 152.21it/s]

[[ 0.          0.         -0.38078091  0.          0.22196782  0.        ]
 [-0.10855882 -0.08712713  0.12255239  0.          0.         -0.33937364]
 [ 0.          0.80865544 -0.04735876  0.          0.          0.        ]]
[[ 0.89561384 -0.27022677  0.          0.          0.30028888  0.        ]
 [ 0.16552375 -0.2225545   0.34582768  0.          0.          0.        ]
 [ 0.         -0.16045208  0.          0.30028888  0.          0.        ]]
-1.7728574541749627
-2.829095825144093


In [23]:
A_hat1, B_hat1 = final_mar_adalasso(X,A_hat_adalasso,B_hat_adalasso,A_cd,B_cd,p,lam1=1,lam2=1,max_iter=500)
print(A_hat1)
print(B_hat1)
for i in range(p):
    print(metrics_dif_kron(A_true[:,i*dim_A:(i+1)*dim_A],B_true[:,i*dim_B:(i+1)*dim_B],A_hat=A_hat1[:,i*dim_A:(i+1)*dim_A],B_hat=B_hat1[:,i*dim_B:(i+1)*dim_B]))


100%|██████████| 500/500 [00:02<00:00, 212.13it/s]

[[ 0.          0.         -0.45900903  0.          0.          0.        ]
 [-0.12145054 -0.0913464   0.07623208  0.          0.         -0.        ]
 [ 0.          0.87150981 -0.02959122  0.          0.          0.        ]]
[[ 0.78924518 -0.20405491  0.          0.          0.05608064  0.        ]
 [ 0.07830667 -0.08286316  0.27919453  0.          0.          0.        ]
 [ 0.         -0.02126067  0.          0.05608064  0.          0.        ]]
-2.4579023338302433
-3.069688958974828


In [27]:
A_hat1, B_hat1 = final_mar_adalasso2(X,A_hat_adalasso,B_hat_adalasso,A_cd,B_cd,p,lam1=1,lam2=1,delta=1e-5,max_iter=500)
print(A_hat1)
print(B_hat1)
for i in range(p):
    print(metrics_dif_kron(A_true[:,i*dim_A:(i+1)*dim_A],B_true[:,i*dim_B:(i+1)*dim_B],A_hat=A_hat1[:,i*dim_A:(i+1)*dim_A],B_hat=B_hat1[:,i*dim_B:(i+1)*dim_B]))


 43%|████▎     | 215/500 [00:01<00:01, 210.70it/s]

[[-0.         -0.         -0.43622788 -0.          0.         -0.        ]
 [-0.08611585 -0.04495405  0.04753916 -0.         -0.         -0.        ]
 [ 0.          0.89331319 -0.         -0.         -0.         -0.        ]]
[[ 0.79291859 -0.23359524  0.          0.11222641  0.11222641  0.11222641]
 [ 0.10922255 -0.14388113  0.30802907  0.11222641  0.11222641  0.11222641]
 [-0.         -0.09119369 -0.          0.11222641  0.11222641  0.11222641]]
-2.1295405436830346
-3.069688958974828
